In [ ]:
!pip install keras_preprocessing -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 1.1 MB/s eta 0:00:00
2023-07-21 19:17:43.867249: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-21 19:17:44.831427: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-07-21 19:17:46.094725: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-07-21 19:17:46.095173: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had ne

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers
import tensorflow.keras.utils as ku
import string, os
import numpy as np
import pandas as pd
from google.colab import drive
drive.mount('/drive')

Mounted at /drive


In [ ]:
tokenizer = Tokenizer()
data = pd.read_csv("/drive/My Drive/Colab Notebooks/dissertation/train.csv")
data['q_a'] = data['Questions'] + ' ' + data['Answers']
all_questions = list(data['q_a'])

def clean_text(txt):
    txt = "".join(t for t in txt if t not in string.punctuation).lower()
    txt = txt.encode("utf8").decode("ascii",'ignore')
    return txt
corpus = [clean_text(x) for x in all_questions]

tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1

In [ ]:
tokenizer = Tokenizer()
def get_sequence_of_tokens(corpus):
    ## tokenization
    tokenizer.fit_on_texts(corpus)
    total_words = len(tokenizer.word_index) + 1

    ## convert data to a token sequence
    input_sequences = []
    for line in corpus:
        token_list = tokenizer.texts_to_sequences([line])[0]
        for i in range(1, len(token_list)):
            n_gram_sequence = token_list[:i+1]
            input_sequences.append(n_gram_sequence)
    return input_sequences, total_words
input_sequences, total_words = get_sequence_of_tokens(corpus)

In [ ]:
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

In [ ]:
# create predictors and label
predictors, label = input_sequences[:,:-1],input_sequences[:,-1]

label = ku.to_categorical(label, num_classes=total_words)

In [ ]:
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
model.add(Bidirectional(LSTM(150, return_sequences = True)))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dense(total_words/2, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(Dense(total_words, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 426, 100)          339800    
                                                                 
 bidirectional (Bidirectiona  (None, 426, 300)         301200    
 l)                                                              
                                                                 
 dropout (Dropout)           (None, 426, 300)          0         
                                                                 
 lstm_1 (LSTM)               (None, 100)               160400    
                                                                 
 dense (Dense)               (None, 1699)              171599    
                                                                 
 dense_1 (Dense)             (None, 3398)              5776600   
                                                        

In [ ]:
history = model.fit(predictors, label, epochs=10, verbose=1)

Epoch 1/10
2609/2609 [==============================] - 215s 78ms/step - loss: 5.7740 - accuracy: 0.1191
Epoch 2/10
2609/2609 [==============================] - 139s 53ms/step - loss: 5.0745 - accuracy: 0.1811
Epoch 3/10
2609/2609 [==============================] - 136s 52ms/step - loss: 4.7003 - accuracy: 0.2200
Epoch 4/10
2609/2609 [==============================] - 134s 51ms/step - loss: 4.3872 - accuracy: 0.2517
Epoch 5/10
2609/2609 [==============================] - 132s 51ms/step - loss: 4.1221 - accuracy: 0.2776
Epoch 6/10
2609/2609 [==============================] - 132s 51ms/step - loss: 3.8989 - accuracy: 0.3033
Epoch 7/10
2609/2609 [==============================] - 132s 51ms/step - loss: 3.7088 - accuracy: 0.3275
Epoch 8/10
2609/2609 [==============================] - 132s 51ms/step - loss: 3.5397 - accuracy: 0.3509
Epoch 9/10
2609/2609 [==============================] - 132s 51ms/step - loss: 3.3878 - accuracy: 0.3716
Epoch 10/10
2609/2609 [==============================] 

In [ ]:
def gen_response(seed_text,next_words):
  response = ''
  for _ in range(next_words):
      token_list = tokenizer.texts_to_sequences([seed_text])[0]
      token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
      predict_x=model.predict(token_list)
      predicted=np.argmax(predict_x,axis=1)
      output_word = ""
      for word, index in tokenizer.word_index.items():
          if index == predicted:
              output_word = word
              break
      seed_text += " " + output_word
      response += " " + output_word

  return response

In [ ]:
# train = pd.read_csv('/drive/My Drive/Colab Notebooks/dissertation/train.csv')
test = pd.read_csv('/drive/My Drive/Colab Notebooks/dissertation/test.csv')

In [ ]:
test['bilstm'] = test.apply(lambda x: gen_response(x['Questions'], 30), axis = 1)

Streaming output truncated to the last 5000 lines.
1/1 [==============================] - 0s 34ms/step


In [ ]:
test.to_csv('/drive/My Drive/Colab Notebooks/dissertation/test.csv',index=False)